# Capstone Project from E. Wesner

This notebook will be mainly used for the capstone project of Coursera Data Science

In [1]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Neighborhoods of Toronto "Dataimport and Cleaning"

**Import the Data from Wikipedia**

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get (URL)
soup = BeautifulSoup(html_data.content, 'html.parser')


In [4]:
#Empty dataframe with cols
neighborhoods_toronto = pd.DataFrame(columns = ["PostalCode","Borough","Neighborhood"])

#from the Soup pick the relevant data
table = soup.find_all("tbody")[0]

#extract the table and values
for row in table.find_all("tr"):
    col = row.find_all("td")
    textlist = [ele.text.strip()for ele in col]
    if textlist:                    #falls in text was drin steht, also nicht [] leer ist
        #print(text)
        for i in range (len(textlist)):
            einzeln = textlist[i]
            postcode = einzeln[0:3]
            schneiden = einzeln[3:].find("(")
            boro = einzeln[3:schneiden+3]   
            neigh = einzeln[schneiden+4:-1]
            neigh = neigh.replace(' / ', ' , ')
            neighborhoods_toronto = neighborhoods_toronto.append({"PostalCode":postcode,"Borough":boro,"Neighborhood":neigh}, 
                                                        ignore_index = True)
neighborhoods_toronto.shape

(180, 3)

In [7]:
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,,Not assigne
1,M2A,,Not assigne
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [5]:
#Not assignet delete
neighborhoods_toronto1=neighborhoods_toronto.replace('Not assigne', np.NaN)
neighborhoods_toronto2=neighborhoods_toronto1.replace('', np.NaN)
neighborhoods_toronto=neighborhoods_toronto2.dropna(subset=["Borough"],axis=0)
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [6]:
neighborhoods_toronto.shape

(103, 3)

In [7]:
#create the new index
neighborhoods_toronto.reset_index(drop=True,inplace=True)
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [9]:
#searching for not assigned neighborhood but a borough
missing_data=neighborhoods_toronto.isnull()
for columns in missing_data.columns.values.tolist():
    print (columns)
    print (missing_data[columns].value_counts())
    print(" ")

PostalCode
False    103
Name: PostalCode, dtype: int64
 
Borough
False    103
Name: Borough, dtype: int64
 
Neighborhood
False    103
Name: Neighborhood, dtype: int64
 


No such data was found

In [11]:
print("The Shape of the DataFrame is: " + str(neighborhoods_toronto.shape))

The Shape of the DataFrame is: (103, 3)


# Part 2   Get the Latitude and Longitude

In [8]:
#Empty dataframe with cols
neighborhoods_coordinates = pd.DataFrame(columns = ["Postal Code","Latitude","Longitude"])
neighborhoods_coordinates.head()

,Postal Code,Latitude,Longitude


**Tried to use geocoder, but I didn't succeed**

In [ ]:
!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

for i in range (neighborhoods_toronto.shape[0]):
    postal_code = neighborhoods_toronto.at[neighborhoods_toronto.index[i],'Postal Code'] 
    print(postal_code)
    #loop until you get the coordinates
    while(lat_lng_coords is None):        
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    neighborhoods_coordinates = neighborhoods_coordinates.append({"Postal Code":postal_code,"Latitude":latitude,"Longitude":longitude}, 
                                                        ignore_index = True)
neighborhoods_coordinates.shape

**This code was created from the Notebook by importing the CSV-file Geospatial_Coordinates.csv**

In [9]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_0041bdc6e7bf463992b36c4279e007e7 = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_0041bdc6e7bf463992b36c4279e007e7 = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

client_0041bdc6e7bf463992b36c4279e007e7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_WvAzxSMMbyyvT7Bo77QzhnGJ2DPckT5a9RkQmE9rCnN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_0041bdc6e7bf463992b36c4279e007e7)

body = client_0041bdc6e7bf463992b36c4279e007e7.get_object(Bucket='courseracapstonedatascience-donotdelete-pr-mgnh2uarlhwkrt',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#Merge the Coordinates with the Neighborhoods: 

neighborhoods_merged = neighborhoods_toronto
neighborhoods_merged = neighborhoods_merged.join(df_data_1.set_index('Postal Code'), on = 'PostalCode')
neighborhoods_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [26]:
neighborhoods_merged.to_csv('GeocoordinatesNeighborhoodsToronto.csv')
print('Daten gesichert')

Daten gesichert


# Part 3   Analyse, Clustering

In [11]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         632 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
   

argon2-cffi-20.1.0   | 47 KB     | ##################################### | 100% 
zstd-1.4.9           | 431 KB    | ##################################### | 100% 
jeepney-0.6.0        | 32 KB     | ##################################### | 100% 
matplotlib-3.3.4     | 6 KB      | ##################################### | 100% 
xz-5.2.5             | 343 KB    | ##################################### | 100% 
beautifulsoup4-4.9.3 | 86 KB     | ##################################### | 100% 
xlrd-2.0.1           | 92 KB     | ##################################### | 100% 
libssh2-1.9.0        | 226 KB    | ##################################### | 100% 
click-7.1.2          | 64 KB     | ##################################### | 100% 
testpath-0.4.4       | 85 KB     | ##################################### | 100% 
jupyterlab_pygments- | 8 KB      | ##################################### | 100% 
ibm-wsrt-py37main-ma | 2 KB      | ##################################### | 100% 
wrapt-1.12.1         | 47 KB

libblas-3.9.0        | 12 KB     | ##################################### | 100% 
secretstorage-3.3.1  | 24 KB     | ##################################### | 100% 
idna-2.10            | 52 KB     | ##################################### | 100% 
tifffile-2021.3.5    | 127 KB    | ##################################### | 100% 
statsmodels-0.12.2   | 11.3 MB   | ##################################### | 100% 
aiohttp-3.7.4        | 632 KB    | ##################################### | 100% 
python_abi-3.7       | 4 KB      | ##################################### | 100% 
psycopg2-2.8.6       | 168 KB    | ##################################### | 100% 
pyodbc-4.0.30        | 71 KB     | ##################################### | 100% 
ca-certificates-2020 | 137 KB    | ##################################### | 100% 
pyqt-impl-5.12.3     | 5.9 MB    | ##################################### | 100% 
mpmath-1.2.1         | 437 KB    | ##################################### | 100% 
pyshp-2.1.0          | 31 KB

backports.functools_ | 8 KB      | ##################################### | 100% 
jsonschema-3.2.0     | 45 KB     | ##################################### | 100% 
protobuf-3.15.6      | 342 KB    | ##################################### | 100% 
more-itertools-8.7.0 | 39 KB     | ##################################### | 100% 
jupyterlab_widgets-1 | 130 KB    | ##################################### | 100% 
readline-8.0         | 281 KB    | ##################################### | 100% 
iniconfig-1.1.1      | 8 KB      | ##################################### | 100% 
json5-0.9.5          | 20 KB     | ##################################### | 100% 
snowballstemmer-2.1. | 57 KB     | ##################################### | 100% 
libpq-13.1           | 2.7 MB    | ##################################### | 100% 
typing-extensions-3. | 8 KB      | ##################################### | 100% 
requests-oauthlib-1. | 21 KB     | ##################################### | 100% 
pyerfa-1.7.2         | 385 K

done
Libraries imported.


In [12]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy   

In [13]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [14]:
# get the Latitude and Longitude from Toronto to create a map
address = 'Toronto,ON'
geolocator = Nominatim(user_agent = "to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The georeaphic coordinates of Toronto are {},{}.'.format(latitude,longitude))


The georeaphic coordinates of Toronto are 43.6534817,-79.3839347.


In [15]:
# reduce the Dataset to Toronto
toronto_data = neighborhoods_merged[neighborhoods_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [16]:
toronto_data.shape

(39, 5)

In [17]:
#Lets see How Torontos Neighborhoods look like
map_toronto = folium.Map(location=[latitude,longitude],zoom_start = 12)

for lat,lng,borough,neighborhood in zip (toronto_data['Latitude'],toronto_data['Longitude'],toronto_data['Borough'],toronto_data['Neighborhood']):
    label = '{},{}'.format (neighborhood,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [18]:
CLIENT_ID = 'WCSAEPVN5OVR2CIQMKXONC04JS505C5CWKJXFYUWLFISIRAD' # your Foursquare ID
CLIENT_SECRET = 'QDG0CIEBA40GVXY2PDJQHIMDCAGHVM3LAYWOZMTNZCGGDUZG' # your Foursquare Secret 
ACCESS_TOKEN = '0N0RLQRFPBTP3JHYLM1S31FXICUDT3UJQIU5O0EZIQXYZAOJ' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WCSAEPVN5OVR2CIQMKXONC04JS505C5CWKJXFYUWLFISIRAD
CLIENT_SECRET:QDG0CIEBA40GVXY2PDJQHIMDCAGHVM3LAYWOZMTNZCGGDUZG


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.head()

Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
The Danforth  East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosedale
Enclave of M5E
St. Jame

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [22]:
print(toronto_venues.shape)


(1581, 7)


In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",16,16,16,16,16,16
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37
Davisville North,7,7,7,7,7,7


In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


# Analyze the Data

In [25]:
# one hot encoding
# each venue category builds a column
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.head())
print(toronto_onehot.shape)

   Yoga Studio  Adult Boutique  Airport  Airport Food Court  Airport Gate  \
0            0               0        0                   0             0   
1            0               0        0                   0             0   
2            0               0        0                   0             0   
3            0               0        0                   0             0   
4            0               0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Thrift / Vintage Store  Tibetan Restaurant  \
0     

In [26]:
#getting mean values of the groups, this indicates the frequency of occurence
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped)
print(toronto_grouped.shape)

                                         Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.000000   
1      Brockton , Parkdale Village , Exhibition Place     0.000000   
2   CN Tower , King and Spadina , Railway Lands , ...     0.000000   
3                                  Central Bay Street     0.016393   
4                                            Christie     0.000000   
5                                Church and Wellesley     0.025974   
6                     Commerce Court , Victoria Hotel     0.000000   
7                                          Davisville     0.000000   
8                                    Davisville North     0.000000   
9                       Dufferin , Dovercourt Village     0.000000   
10                                     Enclave of M4L     0.000000   
11                                     Enclave of M5E     0.010101   
12            First Canadian Place , Underground city     0.000000   
13                  

Create a Function the get the most common venues

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



In [28]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
    

In [29]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Cheese Shop,Pharmacy,Irish Pub
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Pet Store,Stadium,Intersection,Bakery,Italian Restaurant,Restaurant,Climbing Gym
2,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Sculpture Garden
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant,Middle Eastern Restaurant
4,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Restaurant,Baby Store,Candy Store,Athletics & Sports,Coffee Shop


In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1) #for clustering the stringvalues are not usable

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

In [32]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Theater,Gym / Fitness Center,Event Space
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Hotel,Japanese Restaurant,Bubble Tea Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Theater
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Cocktail Bar,Gastropub,Cosmetics Shop,Restaurant,Gym,Moroccan Restaurant,Department Store,Clothing Store
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Trail,Pub,Wine Shop,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Cheese Shop,Pharmacy,Irish Pub


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

*Analyse the clusters*

**What makes the cluster 0 ?**

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,East Toronto,0,Fast Food Restaurant,Food & Drink Shop,Steakhouse,Fish & Chips Shop,Sushi Restaurant,Ice Cream Shop,Liquor Store,Pub,Restaurant,Italian Restaurant
20,Central Toronto,0,Park,Hotel,Breakfast Spot,Food & Drink Shop,Sandwich Place,Department Store,Pizza Place,Convenience Store,Distribution Center,Escape Room
38,East TorontoBusiness reply mail Processing Cen...,0,Light Rail Station,Garden,Brewery,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Restaurant,Butcher,Auto Workshop


Cluster 0: having restaurants and simmilar

In [35]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East YorkEast Toronto,1,Park,Convenience Store,Intersection,Wine Shop,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
21,Central Toronto,1,Jewelry Store,Park,Sushi Restaurant,Trail,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Wine Shop
29,Central Toronto,1,Park,Playground,Tennis Court,Trail,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
33,Downtown Toronto,1,Park,Playground,Trail,Wine Shop,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Cluster 1: Having Parks, Playgrounds, Trails and Event Space

In [36]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Theater,Gym / Fitness Center,Event Space
1,Downtown Toronto,2,Clothing Store,Coffee Shop,Hotel,Japanese Restaurant,Bubble Tea Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Theater
2,Downtown Toronto,2,Coffee Shop,Café,Cocktail Bar,Gastropub,Cosmetics Shop,Restaurant,Gym,Moroccan Restaurant,Department Store,Clothing Store
3,East Toronto,2,Health Food Store,Trail,Pub,Wine Shop,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
4,Downtown Toronto,2,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Cheese Shop,Pharmacy,Irish Pub
5,Downtown Toronto,2,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant,Middle Eastern Restaurant
6,Downtown Toronto,2,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Restaurant,Baby Store,Candy Store,Athletics & Sports,Coffee Shop
7,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Bakery,Gym,Thai Restaurant,Deli / Bodega,Asian Restaurant
8,West Toronto,2,Bakery,Pharmacy,Supermarket,Middle Eastern Restaurant,Bank,Bar,Café,Portuguese Restaurant,Pool,Wine Shop
10,Downtown Toronto,2,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Sporting Goods Shop,Brewery,Italian Restaurant,Restaurant,Scenic Lookout


Cluster 2: mostly Coffe Shops

In [37]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Swim School,Bus Line,Business Service,Wine Shop,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


Cluster 3: Swim School

In [38]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Fast Food Restaurant,Home Service,Garden,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


Cluster 4: Fast Food and Home Service are most popular